### Robustness Benchmark results

In [2]:
# surpress UserWarnings
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)

In [3]:
import mmcv
def read_results_file(jobname, postfix="_results.pkl"):
    folder = "evaluation/benchmark/"
    if postfix == "_tpfp.pkl":
        folder += "tpfp/"
    path = folder + "rob-bm-full" + jobname
    res = mmcv.load(path + postfix)
    return res

In [ ]:
# get keys (corruptions) of result files
for job in ["-2_ff-baseline","_ff-baseline","-2_fb-add","_fb-add","_fb-mod"]:
    res = read_results_file(job)
    last_sevs = list(res.values())[-1].keys()
    print(f'{job}: {res.keys()} severities of last: {last_sevs} \n')

#### Merge the result dicts

In [5]:
def merge_dicts(names, postfix="_results.pkl"):
    dict_1 = read_results_file(names[0], postfix)
    dict_2 = read_results_file(names[1], postfix)
    dict_1.update(dict_2)
    # save to file
    folder = "evaluation/benchmark/merged/"
    path = folder + "rob-bm-merged_" + names[2] + postfix
    mmcv.dump(dict_1, path, file_format='pkl')
    print(f'merged dict: {names[2]}\n {dict_1.keys()} \n')
    return dict_1

In [ ]:
for names in [["-2_ff-baseline","-3_ff-baseline","ff-baseline"],
            ["-2_fb-add","-3_fb-add","fb-add"],
            ["_fb-mod","-3_fb-mod","fb-mod"]]:
    #merge_dicts(names, postfix="_results.pkl")
    merge_dicts(names, postfix='_tpfp.pkl')

merged dict: ff-baseline
 dict_keys(['gaussian_noise', 'shot_noise', 'impulse_noise', 'defocus_blur', 'glass_blur', 'motion_blur', 'zoom_blur', 'snow', 'frost', 'fog', 'brightness', 'contrast', 'elastic_transform', 'pixelate', 'jpeg_compression']) 

merged dict: fb-add
 dict_keys(['gaussian_noise', 'shot_noise', 'impulse_noise', 'defocus_blur', 'glass_blur', 'motion_blur', 'zoom_blur', 'snow', 'frost', 'fog', 'brightness', 'contrast', 'elastic_transform', 'pixelate', 'jpeg_compression']) 

merged dict: fb-mod
 dict_keys(['gaussian_noise', 'shot_noise', 'impulse_noise', 'defocus_blur', 'glass_blur', 'motion_blur', 'zoom_blur', 'snow', 'frost', 'fog', 'brightness', 'contrast', 'elastic_transform', 'pixelate', 'jpeg_compression']) 



#### Process true/false positive lists

In [6]:
import numpy as np
import mmcv

def count_tpfp(tpfp):
    """Count True Positves and False Positives per image 
        in: [class x image x tpfp] 
        out: [image x (count(tp),count(fp))]
    """
    num_imgs = len(tpfp[0]['tp'])
    tp_all = np.empty(num_imgs, dtype=object)
    # go through classes
    for tp_cl in tpfp:
        # go through images
        for (ind,item) in enumerate(tp_cl['tp']):
            if item.shape!=(1,0):
                if tp_all[ind] is None:
                    tp_all[ind] = item[0]
                else:
                    tp_all[ind] = np.concatenate((tp_all[ind], item[0]), axis=0)
    
    tp_count = [(np.sum(tp_img == 1.0), np.sum(tp_img == 0.0)) for tp_img in tp_all]
    return tp_count

def transform_tpfp_keep_class(tpfp):
    """Transform TruePositve-FalsePositive list 
        from: [class x image x tpfp] to: [image x (class_idx, tpfp)]
    """
    tp_all = np.empty(len(tpfp[0]['tp']), dtype=object)
    # go through classes
    for (cl_idx, tp_cl) in enumerate(tpfp):
        # go through images
        for (ind,item) in enumerate(tp_cl['tp']):
            if item.shape!=(1,0):
                item = item[0]
                if tp_all[ind] is None:
                    tp_all[ind] = (cl_idx, item)
                else:
                    tp_all[ind] = (tp_all[ind], (cl_idx, item))

    return tp_all

transform and save tpfp-list for uncorrupted data (severity=0)

In [ ]:
for filename in ["ff-baseline","fb-add","fb-mod"]:
    path_read = f"evaluation/benchmark/merged/rob-bm-merged_{filename}_tpfp.pkl"
    tpfp_old = mmcv.load(path_read)
    tpfp_new = count_tpfp(tpfp_old['gaussian_noise'][0])
    path_out = f"evaluation/benchmark/significance_test/{filename}_tpfp-count_sev0.pkl"
    mmcv.dump(tpfp_new, path_out)

count nr of true and false positives overall

In [7]:
import pandas as pd
data = {}
for filename in ["ff-baseline","fb-add","fb-mod"]:
    path_read = f"evaluation/benchmark/significance_test/{filename}_tpfp-count_sev0.pkl"
    tp_count_list = np.array(mmcv.load(path_read))
    tp_count = np.sum(tp_count_list[:,0])
    fp_count = np.sum(tp_count_list[:,1])
    fp_imgs = np.sum(tp_count_list[:,1]==0)
    data.update({filename: [tp_count, fp_count, fp_imgs]})
pd.DataFrame.from_dict(data=data, orient='index', columns=["tpcount", "fp count", "fp free imgs"])

,tpcount,fp count,fp free imgs
ff-baseline,11105,27675,566
fb-add,10712,13708,1311
fb-mod,10859,16354,1116


-----------
format original TP-FP list:

| 15  |  5   |  20 | 2 | 4952 | n |
|---|---|---|---|-|-|
|corruptions |severities | classes | tp, fp | images |detections|

-----------
format output of detections / ground truth:

| 4952  |  20    |    n     |   5   |
|---|---|---|---|
|images |classes | detected bboxes | bbox + certainty |
----------
### Results Robustness Benchmark
| model | P |  mPC   |  rPC |
|---|--|---|---|
|FF-Baseline | 0.801 | 0.489 | 60,5 % |
|FB-Mod | 0.806 | 0.468 | 57,5 % |
|FB-Add | 0.803 | 0.457 | 56,3 % |

Performance on Clean Data [P] in AP50  
Mean Performance under Corruption [mPC] in AP50  
Relative Performance under Corruption [rPC] in %  

### mAP per Severity
| |ff_base  |fb_mod |fb_add|
|-|---      |--     |   --|
|0|	0.801	|0.806	|0.803|
|1|	0.677	|0.668	|0.659|
|2|	0.594	|0.579	|0.570|
|3|	0.500	|0.477	|0.464|
|4|	0.384	|0.353	|0.340|
|5|	0.288	|0.263	|0.251|
--------
mPC calculation

    mPC = np.mean(results[:15, 1:, :], axis=(0, 1))
-> mean of 1500 APs
| 15  |  5   |  20 | =1500 |
|---|---|---|--|
|corruptions |severities | classes | = total nr.|

In [ ]:
""" 20 classes of PASCAL VOC
 0: aeroplane
 1: bicycle
 2: bird
 3: boat
 4: bottle
 5: bus
 6: car
 7: cat
 8: chair
 9: cow
10: diningtable
11: dog
12: horse
13: motorbike
14: person
15: pottedplant
16: sheep
17: sofa
18: train
19: tvmonitor
"""